## NYTimes_NEWS

In [1]:
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as uReq
import pandas as pd
import nltk as nltk
import re
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
NY_url = "https://www.nytimes.com/topic/subject/artificial-intelligence"

In [3]:
uClient = uReq(NY_url) # requesting the webpage from the internet
news_page = uClient.read() # reading the webpage
uClient.close

<bound method HTTPResponse.close of <http.client.HTTPResponse object at 0x0000027B488EAC08>>

In [4]:
NY_html = bs(news_page, "html.parser")

In [5]:
NY_html

<!DOCTYPE html>

<html lang="en" xmlns:og="http://opengraphprotocol.org/schema/">
<head>
<title data-rh="true">Artificial Intelligence - The New York Times</title>
<meta content="en-US" data-rh="true" itemprop="inLanguage"/><meta content="collection" data-rh="true" id="applicationName" name="applicationName"/><meta content="Artificial Intelligence" data-rh="true" name="nyt-collection:identifier"/><meta content="Artificial Intelligence" data-rh="true" name="CN"/><meta content="topiccollection" data-rh="true" name="nyt-collection:type"/><meta content="topic" data-rh="true" name="CT"/><meta content="Times Topics" data-rh="true" name="nyt-collection:display-name"/><meta content="" data-rh="true" name="nyt-collection:tagline"/><meta content="" data-rh="true" name="nyt-collection:promotional-image"/><meta content="collection" data-rh="true" name="PT"/><meta content="1154675712971" data-rh="true" name="asset_id"/><meta content="Artificial Intelligence" data-rh="true" name="slug"/><meta conten

In [6]:
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as uReq
import pandas as pd
import nltk as nltk
import re
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [7]:
def extract_NYTIMES_html(url):
    try:
        uClient = uReq(url) # requesting the webpage from the internet
        news_page = uClient.read() # reading the webpage
        uClient.close
        NYTIMES_html = bs(news_page, "html.parser")
    except:
        NYTIMES_html = "Webpage Not Responding. Try Including Header"
        
    
    return NYTIMES_html

In [8]:
def extract_NYTIMES_Summary_Contents(NYTIMES_html):
    try:
        summary_contents = NYTIMES_html.findAll("div", {"class": "css-1cp3ece"})
    except:
        summary_contents = "No Summary Retrived"
    return summary_contents

In [9]:
def get_NewsHeadlines(summary_content):
    try:
        news_headline = summary_content.find("h2", {"class": "css-1j9dxys e1xfvim30"}).text
    except:
        news_headline = "No News Headlines"
    return news_headline

In [10]:
def get_NewsLinks(summary_content):
    try:
        news_link = "https://www.nytimes.com/" + summary_content.find("div", {"class": "css-1l4spti"}).a['href']
    except:
        news_link = "No News Link"
    return news_link

In [11]:
def get_NewsDescriptions(news_content):
    try: 
        news_body = news_content.findAll("div", {"class": "css-1fanzo5 StoryBodyCompanionColumn"})
    except:
        news_body = "No News Body"
    
    news_description=""
    
    try:
        for news_desc in news_body:
            paragraphs = news_desc.findAll('p')
            for paragraph in paragraphs:
                paragraph = paragraph.get_text()
                news_description = news_description + paragraph
    except:
        news_description = "No News Description"
    return news_description

In [12]:
def get_PublishedDate(news_content):
    try:
        published_date = news_content.find("time", {"class": "css-1sbuyqj e16638kd3"}).text
    except:
        published_date = "No Published Date"
    return published_date

In [13]:
## Remove Punctuation
def remove_punctuation(text):
    try:
        text_nopunct = ''
        #text_nopunct = re.sub('['+string.punctuation+']', ' ', text)
        text_nopunct = re.sub('[^A-Za-z0-9]+', ' ', text)
    except:
        text_nopunct = "Text punctuation removal not working"
    return text_nopunct

In [14]:
## Convert to lower case
def lowercase_token(tokens): 
    return [w.lower() for w in tokens]

In [15]:
## Remove stopwords
def removestopwords(tokens, stoplist): 
    return [word for word in tokens if word not in stoplist]

In [16]:
## Lematization of words
def process_lemmatize_words(filtered_words):
    lemmatizer=WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in filtered_words]

In [17]:
## Lematization of words verbs
def process_lemmatize_verbs(lemmatize_words):
    lemmatizer=WordNetLemmatizer()
    return [lemmatizer.lemmatize(word, pos='v') for word in lemmatize_words]

In [18]:
def save_News_to_excel(NYTIMES_News_DataFrame,FileName):
    try:
        NYTIMES_News_DataFrame.to_excel(excel_writer= FileName+".xlsx",sheet_name=FileName)
        print("Files Saved Successfully")
    except:
        print("Error occured in saving the file.")

In [19]:
def NYTimes_News(NYTIMES_URL, FileName):
    NYTIMES_News_DataFrame = pd.DataFrame()
    NYTIMES_html = extract_NYTIMES_html(NYTIMES_URL)
    summary_contents = extract_NYTIMES_Summary_Contents(NYTIMES_html)
    News_Headlines = []
    News_Links = []
    Dates_Published = []
    News_Descriptions = []
    for summary_content in summary_contents:
        
        #Get List Of News HeadLines
        news_headline = get_NewsHeadlines(summary_content)
        News_Headlines.append(news_headline)
            
        #Get List Of News Links
        news_link = get_NewsLinks(summary_content)
        News_Links.append(news_link)
        
    for news_link in News_Links:
        
        try:
            newsRes = requests.get(news_link)
            newsRes.encoding = 'utf-8'
            news_content = bs(newsRes.text, "html.parser")
        except:
            news_content = "No content for news"
            
        #Get news Description 
        news_description = get_NewsDescriptions(news_content)
        News_Descriptions.append(news_description)
            
        #Get Published Date
        published_date = get_PublishedDate(news_content)
        Dates_Published.append(published_date)
    mydict = {"News Link": News_Links, "News Headlines": News_Headlines, "Date Published" : Dates_Published, "News Description":News_Descriptions, "Domain": "Miscellaneous", "Company": "Miscellaneous"}    
    NYTIMES_News_DataFrame = pd.concat([NYTIMES_News_DataFrame, pd.DataFrame(mydict, index = None)])
    try:
        NYTIMES_News_DataFrame = NYTIMES_News_DataFrame.rename(columns = {'News Description':'News_Description'})
        NYTIMES_News_DataFrame['News_Description_Cleaned'] = NYTIMES_News_DataFrame['News_Description'].apply(lambda News_Description: remove_punctuation(News_Description))
        ## Tokenize sentences
        tokens = [word_tokenize(sen) for sen in NYTIMES_News_DataFrame.News_Description_Cleaned]
        lower_tokens = [lowercase_token(token) for token in tokens]
        stoplist = stopwords.words('english')
        filtered_words = [removestopwords(sen, stoplist) for sen in lower_tokens]
        lemmatize_words = [process_lemmatize_words(sen) for sen in filtered_words]
        lemmatize_verbs = [process_lemmatize_verbs(sen) for sen in lemmatize_words]
        NYTIMES_News_DataFrame['News_Description_Cleaned'] = [' '.join(sen) for sen in lemmatize_verbs]
        NYTIMES_News_DataFrame['tokens'] = lemmatize_verbs
    except:
        print("There is some problem with text pre-processing")
    save_News_to_excel(NYTIMES_News_DataFrame, FileName)

In [20]:
NYTimes_News("https://www.nytimes.com/topic/subject/artificial-intelligence", "NYTIMES_NEWS")

Files Saved Successfully
